## Web Scarping using Selenium

This notebook contains the code to scrape data for 1000 popular books on www.gutenberg.com

Importing the required libraries

In [20]:
import pandas as pd

Setting up selenium to browser and driver. I am using Firefox for the browser. The driver used can be download from [here](https://github.com/mozilla/geckodriver/releases)

In [21]:
from selenium import webdriver 
from selenium.webdriver.firefox.service import Service
s=Service("D:\Setups\geckodriver-v0.30.0-win64\geckodriver.exe")
driver = webdriver.Firefox(service=s)

Connecting to the website.

In [23]:
driver.get('https://www.gutenberg.org/ebooks/search/?sort_order=downloads')

Finding the required class to scrape for data regarding the books

In [24]:
books=driver.find_elements_by_class_name('booklink')
len(books)
print(books[-1].text)


Grimms' Fairy Tales
Jacob Grimm and Wilhelm Grimm
14420 downloads


C:\Users\SHLOKR~1\AppData\Local\Temp/ipykernel_27336/2740133046.py:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  books=driver.find_elements_by_class_name('booklink')


Extracting the author and total download information

In [25]:
name=books[0].find_elements_by_class_name('title')[0].text
author=books[0].find_elements_by_class_name('subtitle')[0].text
#total_downloads=books[0].find_elements_by_class_name('extra')[0].text
print(name)
print(author)
#print(total_downloads)

Frankenstein; Or, The Modern Prometheus
Mary Wollstonecraft Shelley


D:\Python\lib\site-packages\selenium\webdriver\remote\webelement.py:463: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


Script to extract data from the page and iterate to next page.

In [26]:

count = 0
index=0
df=pd.DataFrame(columns=['Name', 'Author', 'Total Downloads'])
name_list=[]
author_list=[]
total_downloads_list=[]
while True:
    if count==40:
        break
    count+=1
    print('page ',count)
    books = driver.find_elements_by_class_name('booklink')
    
    for book in books:
        try:
            name_list.append(book.find_elements_by_class_name('title')[0].text)
            try:
                author_list.append(book.find_elements_by_class_name('subtitle')[0].text)
               # df['Author'].append(author)
            except:
                author_list.append('Not availbale')
               # df['Author'].append(author)
            try:
                total_downloads_list.append(book.find_elements_by_class_name('extra')[0].text)
                #df['Total Downloads'].append(total_downloads)
            except:
                total_downloads_list.append('Not availbale')
               # df['Total Downloads'].append(total_downloads)
            
            
            index+=1
            
        except:
            pass
    try:    
        driver.find_elements_by_class_name('statusline')[0].find_elements_by_tag_name('a')[-1].click()
    except:
        break
    print('|'*100)
df["Name"]=name_list
df["Author"]=author_list
df["Total Downloads"]=total_downloads_list
print(df)


page  1


C:\Users\SHLOKR~1\AppData\Local\Temp/ipykernel_27336/2620929844.py:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  books = driver.find_elements_by_class_name('booklink')
C:\Users\SHLOKR~1\AppData\Local\Temp/ipykernel_27336/2620929844.py:36: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_class_name('statusline')[0].find_elements_by_tag_name('a')[-1].click()
D:\Python\lib\site-packages\selenium\webdriver\remote\webelement.py:358: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  2
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  3
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  4
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  5
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  6
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  7
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  8
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  9
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
page  10
||||||||||||||||||

Saving the dataframe to csv file

In [27]:
df.to_csv('popular_books.csv')